<a href="https://colab.research.google.com/github/furrutiav/data-mining-2022/blob/main/Hitos/H3/BETO_embeddings_tweet_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import pickle

In [6]:
%%capture
!pip install transformers
!pip install datasets
!pip install bertviz
!pip install torch

In [7]:
from transformers import BertTokenizer, BertModel
import torch

In [8]:
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

--2022-07-03 03:23:46--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz.2’

pytorch_weights.tar 100%[===================>] 390.88M  5.06MB/s    in 54s     

2022-07-03 03:24:41 (7.25 MB/s) - ‘pytorch_weights.tar.gz.2’ saved [409871727/409871727]

--2022-07-03 03:24:41--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242120 (236K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 236.45K   408KB/s    in 

In [9]:
from transformers import BertForMaskedLM, BertTokenizer

In [10]:
beto_model = BertModel.from_pretrained("pytorch/")
beto_tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
e = beto_model.eval()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
df_es_train = pd.read_pickle("/content/drive/MyDrive/df_es_train.pickle")

In [12]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [24]:
def sentence_clf_output(text):
    # retorna el SequenceClassifierOutput dado un tweet
    text = preprocess(text)
    tokens = beto_tokenizer.tokenize(text)
    tokens = ['[CLS]'] + tokens
    attn_mask = [1 for token in tokens]
    seg_ids = [0 for _ in range(len(tokens))]
    token_ids = beto_tokenizer.convert_tokens_to_ids(tokens)

    token_ids = torch.tensor(token_ids).unsqueeze(0)
    attn_mask = torch.tensor(attn_mask).unsqueeze(0)
    seg_ids   = torch.tensor(seg_ids).unsqueeze(0)

    output = beto_model(token_ids, attention_mask = attn_mask, token_type_ids = seg_ids)
    cls_output = output.last_hidden_state[:, 0].detach().numpy().reshape(1,768)
    return cls_output

In [26]:
input_0 = df_es_train.iloc[0]["text"]
sentence_clf_output(input_0);

In [16]:
import os

In [17]:
folder = "beto_emoji"

In [28]:
def guardar(y_list,idx,conjunto):
    embedding_types = "first_tok_embedding"

    arr = np.concatenate(y_list, axis=0)
    np.save(os.path.join(folder,'vec_{}_{}_{}'.format(conjunto,embedding_types,idx)), arr)

In [ ]:
y_clf_obj = []
length = len(df_es_train)
save_rate = 50

for i, texto in enumerate(df_es_train['text']):
    idx = 0
    clf_obj = sentence_clf_output(texto)
    y_clf_obj.append(clf_obj)
    if (i%(save_rate)==0 and i!=0) or i==length-1:
        guardar(y_clf_obj,idx,'train')
        idx += 1
        y_clf_obj = []
        print('archivo guardado: porcentaje = {}%'.format(100*(i+1)/length))

archivo guardado: porcentaje = 0.0627105722647124%
archivo guardado: porcentaje = 0.12419152546541082%
archivo guardado: porcentaje = 0.18567247866610925%
archivo guardado: porcentaje = 0.24715343186680766%
archivo guardado: porcentaje = 0.30863438506750607%
archivo guardado: porcentaje = 0.37011533826820453%
archivo guardado: porcentaje = 0.43159629146890294%
archivo guardado: porcentaje = 0.49307724466960134%
archivo guardado: porcentaje = 0.5545581978702998%
archivo guardado: porcentaje = 0.6160391510709982%
archivo guardado: porcentaje = 0.6775201042716966%
archivo guardado: porcentaje = 0.7390010574723951%
archivo guardado: porcentaje = 0.8004820106730934%
archivo guardado: porcentaje = 0.8619629638737919%
archivo guardado: porcentaje = 0.9234439170744904%
archivo guardado: porcentaje = 0.9849248702751887%
archivo guardado: porcentaje = 1.046405823475887%
archivo guardado: porcentaje = 1.1078867766765856%
archivo guardado: porcentaje = 1.169367729877284%
archivo guardado: porcenta